# Session 3.1: Enhanced RAG Pipeline

## Overview

In this notebook, we'll enhance our RAG system with advanced retrieval techniques:
- **BM25 Keyword Search**: When exact matches matter
- **Hybrid Search**: Combining semantic + keyword search with RRF
- **Cross-Encoder Reranking**: Two-stage retrieval for better precision
- **Confidence Thresholds**: Know when to say "I don't know"
- **Source Citations**: Proper attribution in answers

**Learning Objectives:**
- Understand when BM25 beats semantic search
- Implement Reciprocal Rank Fusion for hybrid search
- Apply cross-encoder reranking to improve precision
- Build a complete enhanced RAG pipeline

**Prerequisites:**
- Completed Session 2 notebooks (embeddings, ChromaDB, basic RAG)

## The Enhanced RAG Pipeline

```
BASIC RAG (Day 2):
Query → Semantic Search → Top-K → LLM → Answer

ENHANCED RAG (Day 3):
Query → ┬─ Semantic Search ─┬→ Hybrid Fusion → Reranking → Top-K → LLM → Answer
        └─ BM25 Search ─────┘                    ↑           with Citations
                                          Cross-Encoder
```

**Why enhance?**
- Semantic search misses exact keyword matches (IDs, names, codes)
- First-stage retrieval optimizes for recall, not precision
- Reranking trades speed for accuracy on the final results

## Section 1: Setup and Installation

In [1]:
# Install required packages
# Note: Qwen3 reranker requires transformers>=4.51.0
!pip install chromadb openai rank-bm25 sentence-transformers requests -q
!pip install transformers>=4.51.0 -q

print("✓ Packages installed")

✓ Packages installed


In [2]:
import chromadb
from chromadb.config import Settings
import json
import re
import numpy as np
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass
from openai import OpenAI
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
import os
import requests

print(f"ChromaDB version: {chromadb.__version__}")
print("✓ All imports successful")

ChromaDB version: 1.3.7
✓ All imports successful


In [3]:
# Configure OpenRouter API
from google.colab import userdata
os.environ['OPENROUTER_API_KEY'] = userdata.get('OPENROUTER_API_KEY')

OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY', None)

if not OPENROUTER_API_KEY:
    raise ValueError("Please set your OpenRouter API key in Colab secrets.")

# Initialize OpenAI client with OpenRouter endpoint
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

# Models to use
EMBEDDING_MODEL = "qwen/qwen3-embedding-8b"
LLM_MODEL = "anthropic/claude-sonnet-4.5"

print(f"✓ OpenRouter client configured")
print(f"✓ Embedding model: {EMBEDDING_MODEL}")
print(f"✓ LLM model: {LLM_MODEL}")

✓ OpenRouter client configured
✓ Embedding model: qwen/qwen3-embedding-8b
✓ LLM model: anthropic/claude-sonnet-4.5


## Section 2: Load Documents

We'll use two documents for this workshop:
- **tecaj.md**: Croatian Moodle course transcripts (~6700 lines)
- **moodle.md**: English Moodle 5.0 documentation (~1500 lines)

### Upload Instructions

1. Download the data files from the workshop repository:
   - `tecaj.md` (Croatian)
   - `moodle.md` (English)

2. In Colab, click the **folder icon** (📁) in the left sidebar

3. Click the **upload icon** (⬆️) and select both files

4. The files will appear in `/content/` directory

Alternatively, you can mount Google Drive if the files are stored there.

In [4]:
# Upload files using Colab's file upload widget
from google.colab import files
import os

# Check if files already exist (e.g., from previous upload)
tecaj_path = '/content/tecaj.md'
moodle_path = '/content/moodle.md'

files_needed = []
if not os.path.exists(tecaj_path):
    files_needed.append('tecaj.md')
if not os.path.exists(moodle_path):
    files_needed.append('moodle.md')

if files_needed:
    print(f"Please upload: {', '.join(files_needed)}")
    print("Click 'Choose Files' below and select the files.\n")
    uploaded = files.upload()

    # Move uploaded files to expected paths
    for filename in uploaded.keys():
        if filename == 'tecaj.md':
            with open(tecaj_path, 'wb') as f:
                f.write(uploaded[filename])
        elif filename == 'moodle.md':
            with open(moodle_path, 'wb') as f:
                f.write(uploaded[filename])
    print("\n✓ Files uploaded successfully!")
else:
    print("✓ Files already present in /content/")

✓ Files already present in /content/


In [5]:
# Load the documents
with open('/content/tecaj.md', 'r', encoding='utf-8') as f:
    TECAJ_CONTENT = f.read()

with open('/content/moodle.md', 'r', encoding='utf-8') as f:
    MOODLE_CONTENT = f.read()

print(f"✓ Loaded tecaj.md: {len(TECAJ_CONTENT):,} characters, {len(TECAJ_CONTENT.splitlines()):,} lines")
print(f"✓ Loaded moodle.md: {len(MOODLE_CONTENT):,} characters, {len(MOODLE_CONTENT.splitlines()):,} lines")

# Preview first 500 characters of each
print("\n--- tecaj.md preview ---")
print(TECAJ_CONTENT[:500] + "...")
print("\n--- moodle.md preview ---")
print(MOODLE_CONTENT[:500] + "...")

✓ Loaded tecaj.md: 235,490 characters, 6,774 lines
✓ Loaded moodle.md: 114,342 characters, 1,506 lines

--- tecaj.md preview ---
page\_url,text\_after\_transcript

https://usavrsavanje.loomen.carnet.hr/mod/book/view.php?id=106877&chapterid=40940,"O tečaju: Tehnička pitanja | usavrsavanje

**Pokaži transkript**

Postoji nekoliko načina za kretanje unutar Moodle-a. Navigacijska traka pri vrhu ekrana omogućava povratak na određene dijelove sustava koji su hijerarhijski viši od one na kojoj se trenutno nalazimo, primjerice povratak na početnu stranicu

tečaja ili naslovnicu Moodle-a.

Blok Navigacija omogućava brz i jednostav...

--- moodle.md preview ---
#### [**https://docs.moodle.org/500/en/Main\_page**](https://docs.moodle.org/500/en/Main_page)

# **Get started**

# [**Teacher quick guide - MoodleDocs**](https://docs.moodle.org/500/en/Teacher_quick_guide)

In Moodle, a teacher has responsibility for the materials in their own course. They often also manage enrolments and are able to ch

In [6]:
# Chunking function
def chunk_text(text: str, chunk_size: int = 300, overlap: int = 50, source_prefix: str = "") -> List[Dict]:
    """
    Chunk text by sections (split on ---) or by word count.

    Args:
        text: Document text to chunk
        chunk_size: Maximum words per chunk
        overlap: Word overlap between chunks
        source_prefix: Prefix for chunk IDs to ensure uniqueness

    Returns list of dicts with 'text' and 'chunk_id'.
    """
    # First try to split by section markers
    sections = text.split('---')

    chunks = []
    chunk_id = 0

    for section in sections:
        section = section.strip()
        if not section:
            continue

        words = section.split()

        # If section is small enough, keep as one chunk
        if len(words) <= chunk_size:
            chunks.append({
                'text': section,
                'chunk_id': f'{source_prefix}chunk_{chunk_id}',
                'word_count': len(words)
            })
            chunk_id += 1
        else:
            # Split large sections
            for i in range(0, len(words), chunk_size - overlap):
                chunk_words = words[i:i + chunk_size]
                chunk_str = ' '.join(chunk_words)
                if chunk_str.strip():
                    chunks.append({
                        'text': chunk_str,
                        'chunk_id': f'{source_prefix}chunk_{chunk_id}',
                        'word_count': len(chunk_words)
                    })
                    chunk_id += 1

    return chunks

# Process documents with unique prefixes
tecaj_chunks = chunk_text(TECAJ_CONTENT, source_prefix="tecaj_")
moodle_chunks = chunk_text(MOODLE_CONTENT, source_prefix="moodle_")

# Add metadata
for chunk in tecaj_chunks:
    chunk['source'] = 'tecaj.md'
    chunk['language'] = 'hr'

for chunk in moodle_chunks:
    chunk['source'] = 'moodle.md'
    chunk['language'] = 'en'

all_chunks = tecaj_chunks + moodle_chunks

print(f"✓ Created {len(tecaj_chunks)} chunks from tecaj.md (Croatian)")
print(f"✓ Created {len(moodle_chunks)} chunks from moodle.md (English)")
print(f"✓ Total chunks: {len(all_chunks)}")
print(f"\nSample IDs: {[c['chunk_id'] for c in all_chunks[:3]]} ... {[c['chunk_id'] for c in all_chunks[-3:]]}")

✓ Created 134 chunks from tecaj.md (Croatian)
✓ Created 65 chunks from moodle.md (English)
✓ Total chunks: 199

Sample IDs: ['tecaj_chunk_0', 'tecaj_chunk_1', 'tecaj_chunk_2'] ... ['moodle_chunk_62', 'moodle_chunk_63', 'moodle_chunk_64']


## Section 3: Baseline RAG with ChromaDB

First, let's set up a basic semantic search with ChromaDB (as we did in Day 2).

In [7]:
class OpenRouterEmbeddingFunction:
    """
    Custom embedding function for ChromaDB using OpenRouter.
    Implements both __call__ (for documents) and embed_query (for queries).
    """
    def __init__(self, api_key: str, model: str = EMBEDDING_MODEL):
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key
        )
        self.model = model

    def __call__(self, input: List[str]) -> List[List[float]]:
        """Generate embeddings for documents."""
        response = self.client.embeddings.create(
            input=input,
            model=self.model
        )
        return [item.embedding for item in response.data]

    def embed_query(self, input: List[str]) -> List[List[float]]:
        """Generate embeddings for queries (same as documents for this model)."""
        return self.__call__(input)

# Create embedding function
embedding_function = OpenRouterEmbeddingFunction(OPENROUTER_API_KEY)

# Initialize ChromaDB
chroma_client = chromadb.Client()

# Delete existing collection if it exists
try:
    chroma_client.delete_collection(name="workshop_docs")
except:
    pass

# Create collection
collection = chroma_client.create_collection(
    name="workshop_docs",
    embedding_function=embedding_function,
    metadata={"hnsw:space": "cosine"}
)

# Add documents
collection.add(
    documents=[c['text'] for c in all_chunks],
    ids=[c['chunk_id'] for c in all_chunks],
    metadatas=[{'source': c['source'], 'language': c['language']} for c in all_chunks]
)

print(f"✓ Indexed {collection.count()} chunks in ChromaDB")

✓ Indexed 199 chunks in ChromaDB


In [8]:
def semantic_search(query: str, collection, top_k: int = 5) -> List[Dict]:
    """
    Basic semantic search using ChromaDB.
    """
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )

    output = []
    for i in range(len(results['documents'][0])):
        output.append({
            'chunk_id': results['ids'][0][i],
            'text': results['documents'][0][i],
            'metadata': results['metadatas'][0][i],
            'distance': results['distances'][0][i],
            'similarity': 1 - results['distances'][0][i]
        })

    return output

# Test baseline
test_query = "Što je Workshop aktivnost?"
baseline_results = semantic_search(test_query, collection)

print(f"Query: {test_query}")
print("\nBaseline Semantic Search Results:")
print("=" * 60)
for i, r in enumerate(baseline_results[:3], 1):
    print(f"\n[{i}] Similarity: {r['similarity']:.3f} | Source: {r['metadata']['source']}")
    print(f"    {r['text'][:150]}...")

Query: Što je Workshop aktivnost?

Baseline Semantic Search Results:

[1] Similarity: 0.382 | Source: tecaj.md
    biti potpuno odvojene ili međusobno vidljive. Odabirom ""Pohrani i vrati se na stranicu e-kolegija"", chat ste uspješno dodali u tečaj. Nakon prijave ...

[2] Similarity: 0.380 | Source: moodle.md
    Quiz navigation block:** ### Teacher view **Teachers should preview the quiz to ensure it displays as desired for students:** **Grades can be viewed e...

[3] Similarity: 0.377 | Source: moodle.md
    this opens more options.)** 9. **Availability gives you the option to allow students to start peer assessing as soon as the submission deadline is ove...


## Section 4: BM25 Keyword Search

### When BM25 Beats Semantic Search

Semantic search finds similar *meanings*, but sometimes you need exact *matches*:
- Technical IDs: "order #12345", "user ID 789"
- Named entities: "AAI@Edu.hr", "loomen.carnet.hr"
- Specific terminology: "Ocjenjivanje naslijepo"
- Rare words that embedding models might not handle well

**BM25** (Best Matching 25) is a keyword-based ranking function:
- Counts term frequency in documents
- Weights by inverse document frequency (rare terms matter more)
- Normalizes by document length

In [9]:
class BM25Retriever:
    """
    BM25 keyword-based retriever.
    """

    def __init__(self, chunks: List[Dict]):
        """
        Initialize BM25 index.

        Args:
            chunks: List of chunk dicts with 'text', 'chunk_id', 'metadata'
        """
        self.chunks = chunks

        # Tokenize all chunks
        self.tokenized_chunks = [self._tokenize(c['text']) for c in chunks]

        # Build BM25 index
        self.bm25 = BM25Okapi(self.tokenized_chunks)

        print(f"✓ BM25 index built with {len(chunks)} documents")

    def _tokenize(self, text: str) -> List[str]:
        """
        Simple tokenization: lowercase, remove punctuation, split.
        """
        text = text.lower()
        # Keep alphanumeric and spaces
        text = re.sub(r'[^\w\s]', ' ', text)
        return text.split()

    def search(self, query: str, top_k: int = 10) -> List[Dict]:
        """
        Search using BM25.

        Returns list of results with 'text', 'chunk_id', 'bm25_score'.
        """
        tokenized_query = self._tokenize(query)
        scores = self.bm25.get_scores(tokenized_query)

        # Get top-k indices
        top_indices = sorted(
            range(len(scores)),
            key=lambda i: scores[i],
            reverse=True
        )[:top_k]

        results = []
        for idx in top_indices:
            chunk = self.chunks[idx]
            results.append({
                'chunk_id': chunk['chunk_id'],
                'text': chunk['text'],
                'metadata': {'source': chunk['source'], 'language': chunk['language']},
                'bm25_score': float(scores[idx])
            })

        return results


# Build BM25 index
bm25_retriever = BM25Retriever(all_chunks)

✓ BM25 index built with 199 documents


In [10]:
# Test BM25 on a query where keywords matter
keyword_query = "Ocjenjivanje naslijepo Sudionik"

bm25_results = bm25_retriever.search(keyword_query, top_k=5)

print(f"Query: {keyword_query}")
print("\nBM25 Keyword Search Results:")
print("=" * 60)
for i, r in enumerate(bm25_results[:3], 1):
    print(f"\n[{i}] BM25 Score: {r['bm25_score']:.3f} | Source: {r['metadata']['source']}")
    print(f"    {r['text'][:150]}...")

Query: Ocjenjivanje naslijepo Sudionik

BM25 Keyword Search Results:

[1] BM25 Score: 12.855 | Source: tecaj.md
    za opciju Zahtijevaj predaju zadaće za sve članove grupe odaberete NE bit će dovoljno da samo jedan član grupe preda zadaću i klikne gumb za predaju. ...

[2] BM25 Score: 12.576 | Source: tecaj.md
    htjeli ili Automatski do polazne dok se ne postigne prolazna ocjena na zadaći. U padajućem izborniku Najveći dopušteni broj predanih zadaća možete odr...

[3] BM25 Score: 6.628 | Source: tecaj.md
    – polaznicima služi kao pregled svih tečajeva na koje su upisani uz prikaz trenutne vrijednosti konačne ocjene za svaki tečaj. Nastavniku se ovdje pri...


In [11]:
# Compare BM25 vs Semantic on the same query
comparison_query = "AAI@Edu.hr loomen.carnet.hr"

print(f"Query: {comparison_query}")
print("\n" + "="*70)
print("SEMANTIC SEARCH:")
print("="*70)

semantic_results = semantic_search(comparison_query, collection, top_k=3)
for i, r in enumerate(semantic_results, 1):
    print(f"[{i}] Sim: {r['similarity']:.3f} | {r['text'][:100]}...")

print("\n" + "="*70)
print("BM25 KEYWORD SEARCH:")
print("="*70)

bm25_results = bm25_retriever.search(comparison_query, top_k=3)
for i, r in enumerate(bm25_results, 1):
    print(f"[{i}] Score: {r['bm25_score']:.3f} | {r['text'][:100]}...")

print("\n💡 Note: BM25 finds exact matches for technical terms like URLs and IDs!")

Query: AAI@Edu.hr loomen.carnet.hr

SEMANTIC SEARCH:
[1] Sim: 0.585 | |...
[2] Sim: 0.535 | nastaviti koristiti svoj demo tečaj nakon završetka tečaja Moodle dizajner, a CARNetov sam korisnik ...
[3] Sim: 0.530 | na Vaše ime u gornjem desnom kutu ekrana i odabirom opcije ''Moj profil'' iz padajućeg izbornika, na...

BM25 KEYWORD SEARCH:
[1] Score: 12.420 | nastaviti koristiti svoj demo tečaj nakon završetka tečaja Moodle dizajner, a CARNetov sam korisnik ...
[2] Score: 12.311 | na Vaše ime u gornjem desnom kutu ekrana i odabirom opcije ''Moj profil'' iz padajućeg izbornika, na...
[3] Score: 12.282 | edukacija te prikaz dobivene značke i poveznica za preuzimanje uvjerenja za svaku pojedinu edukaciju...

💡 Note: BM25 finds exact matches for technical terms like URLs and IDs!


## Utils: HTML function for displaying results

In [ ]:
def display_results(question: str, result: Dict, title: str = None, show_answer: str = None):
    """Display results with collapsible retrieved chunks (dark theme)."""
    confident_icon = "✅" if result['confident'] else "❌"
    border_color = "#27ae60" if result['confident'] else "#c0392b"
    header_bg = "#1e3a1e" if result['confident'] else "#3a1e1e"

    # Build results table - NO TRUNCATION
    rows = ""
    for i, r in enumerate(result.get('results', [])[:5], 1):
        score = r.get('rerank_score', 0)
        text = r.get('text', '').replace('<', '&lt;').replace('>', '&gt;')
        source = r.get('metadata', {}).get('source', 'unknown')
        chunk_id = r.get('chunk_id', '')
        score_color = "#27ae60" if score > 0.5 else "#f39c12" if score > 0.1 else "#c0392b"
        rows += f"""
        <tr style="background:#2d2d2d;">
            <td style="padding:10px;border-bottom:1px solid #444;color:#fff;vertical-align:top;"><b>{i}</b></td>
            <td style="padding:10px;border-bottom:1px solid #444;vertical-align:top;">
                <span style="background:{score_color};color:white;padding:4px 8px;border-radius:4px;font-weight:bold;">{score:.3f}</span>
            </td>
            <td style="padding:10px;border-bottom:1px solid #444;font-size:11px;color:#888;vertical-align:top;">{source}<br><span style="color:#666;">{chunk_id}</span></td>
            <td style="padding:10px;border-bottom:1px solid #444;font-size:12px;color:#ccc;line-height:1.5;">{text}</td>
        </tr>"""

    # Optional answer section
    answer_html = ""
    if show_answer:
        answer_text = show_answer.replace('<', '&lt;').replace('>', '&gt;').replace('\n', '<br>')
        answer_html = f"""
        <details style="margin-top:15px;" open>
            <summary style="cursor:pointer;font-weight:bold;color:#9b59b6;padding:8px;background:#2d2d2d;border-radius:4px;">
                💬 Generated Answer
            </summary>
            <div style="background:#2d2d2d;padding:15px;margin-top:10px;border-radius:4px;color:#fff;line-height:1.6;">
                {answer_text}
            </div>
        </details>
        """

    display_title = title or question

    html = f"""
    <div style="border:2px solid {border_color};border-radius:8px;margin:10px 0;background:#1a1a1a;font-family:system-ui,-apple-system,sans-serif;">
        <div style="background:{header_bg};padding:15px;border-radius:6px 6px 0 0;">
            <h4 style="margin:0;color:#fff;">{confident_icon} {display_title}</h4>
        </div>
        <div style="padding:15px;">
            <p style="color:#fff;margin:5px 0;"><b>Confident:</b> {result['confident']} | <b>Max Score:</b> {result['max_score']:.3f}</p>
            <p style="color:#888;font-style:italic;margin:5px 0;">{result['message']}</p>
            {answer_html}
            <details style="margin-top:15px;">
                <summary style="cursor:pointer;font-weight:bold;color:#3498db;padding:8px;background:#2d2d2d;border-radius:4px;">
                    📄 Retrieved Sources ({len(result.get('results', []))} chunks)
                </summary>
                <table style="width:100%;border-collapse:collapse;margin-top:10px;">
                    <tr style="background:#1a1a1a;">
                        <th style="padding:10px;text-align:left;color:#3498db;border-bottom:2px solid #3498db;width:30px;">#</th>
                        <th style="padding:10px;text-align:left;color:#3498db;border-bottom:2px solid #3498db;width:70px;">Score</th>
                        <th style="padding:10px;text-align:left;color:#3498db;border-bottom:2px solid #3498db;width:100px;">Source</th>
                        <th style="padding:10px;text-align:left;color:#3498db;border-bottom:2px solid #3498db;">Full Text</th>
                    </tr>
                    {rows}
                </table>
            </details>
        </div>
    </div>
    """
    display(HTML(html))


def display_rank_documents(
    query: str,
    semantic_results: List[Dict],
    bm25_results: List[Dict],
    hybrid_results: List[Dict],
    reranked_results: List[Dict],
    top_k: int = 10
):
    """
    Display document rankings across all search stages in an HTML table (dark theme).

    Args:
        query: The search query
        semantic_results: Results from semantic search (with 'similarity' key)
        bm25_results: Results from BM25 search (with 'bm25_score' key)
        hybrid_results: Results from hybrid/RRF fusion (with 'rrf_score' key)
        reranked_results: Results from cross-encoder reranking (with 'rerank_score' key)
        top_k: Number of results to display per stage
    """
    # Build rank lookup dicts
    sem_ids = {r['chunk_id']: (i+1, r.get('similarity', 0)) for i, r in enumerate(semantic_results[:top_k])}
    bm25_ids = {r['chunk_id']: (i+1, r.get('bm25_score', 0)) for i, r in enumerate(bm25_results[:top_k])}
    hybrid_ids = {r['chunk_id']: (i+1, r.get('rrf_score', 0)) for i, r in enumerate(hybrid_results[:top_k])}
    rerank_ids = {r['chunk_id']: (i+1, r.get('rerank_score', 0)) for i, r in enumerate(reranked_results[:top_k])}

    # Find overlaps
    sem_bm25_overlap = set(sem_ids.keys()) & set(bm25_ids.keys())

    # Build lookup for chunk text and metadata
    chunk_data = {}
    for r in semantic_results + bm25_results + hybrid_results + reranked_results:
        if r['chunk_id'] not in chunk_data:
            chunk_data[r['chunk_id']] = {
                'text': r.get('text', '')[:150] + '...',
                'source': r.get('metadata', {}).get('source', 'unknown')
            }

    def format_rank_cell(rank_info, score_type='similarity'):
        """Format a rank cell with score badge."""
        if rank_info is None:
            return '<td style="padding:8px;border-bottom:1px solid #444;color:#666;text-align:center;">-</td>'
        rank, score = rank_info
        if score_type == 'rerank':
            color = "#27ae60" if score > 0.5 else "#f39c12" if score > 0.1 else "#c0392b"
        else:
            color = "#3498db"
        return f'''<td style="padding:8px;border-bottom:1px solid #444;text-align:center;">
            <span style="color:#fff;font-weight:bold;">#{rank}</span><br>
            <span style="background:{color};color:white;padding:2px 6px;border-radius:3px;font-size:10px;">{score:.3f}</span>
        </td>'''

    def format_movement(hybrid_rank, rerank_rank):
        """Show rank movement from hybrid to rerank."""
        if hybrid_rank is None or rerank_rank is None:
            return ''
        diff = hybrid_rank - rerank_rank
        if diff > 0:
            return f'<span style="color:#27ae60;">↑{diff}</span>'
        elif diff < 0:
            return f'<span style="color:#c0392b;">↓{abs(diff)}</span>'
        return '<span style="color:#888;">=</span>'

    # Build table rows based on reranked order (final ranking)
    rows = ""
    for i, r in enumerate(reranked_results[:top_k], 1):
        cid = r['chunk_id']
        in_both = "⭐" if cid in sem_bm25_overlap else ""
        data = chunk_data.get(cid, {'text': '', 'source': 'unknown'})
        text = data['text'].replace('<', '&lt;').replace('>', '&gt;')

        sem_info = sem_ids.get(cid)
        bm25_info = bm25_ids.get(cid)
        hybrid_info = hybrid_ids.get(cid)
        rerank_info = rerank_ids.get(cid)

        hybrid_rank = hybrid_info[0] if hybrid_info else None
        rerank_rank = rerank_info[0] if rerank_info else None
        movement = format_movement(hybrid_rank, rerank_rank)

        rows += f'''
        <tr style="background:#2d2d2d;">
            <td style="padding:8px;border-bottom:1px solid #444;color:#fff;text-align:center;font-weight:bold;">{i} {in_both}</td>
            <td style="padding:8px;border-bottom:1px solid #444;font-size:11px;color:#888;">{cid}<br><span style="color:#666;">{data['source']}</span></td>
            {format_rank_cell(sem_info, 'similarity')}
            {format_rank_cell(bm25_info, 'bm25')}
            {format_rank_cell(hybrid_info, 'rrf')}
            {format_rank_cell(rerank_info, 'rerank')}
            <td style="padding:8px;border-bottom:1px solid #444;text-align:center;">{movement}</td>
            <td style="padding:8px;border-bottom:1px solid #444;font-size:11px;color:#ccc;">{text}</td>
        </tr>'''

    # Stats
    total_unique = len(set(sem_ids.keys()) | set(bm25_ids.keys()) | set(hybrid_ids.keys()) | set(rerank_ids.keys()))

    html = f"""
    <div style="border:2px solid #3498db;border-radius:8px;margin:10px 0;background:#1a1a1a;font-family:system-ui,-apple-system,sans-serif;">
        <div style="background:#1e3a5e;padding:15px;border-radius:6px 6px 0 0;">
            <h4 style="margin:0;color:#fff;">🔍 Document Rankings Across Search Stages</h4>
            <p style="margin:5px 0 0 0;color:#aaa;font-size:12px;">Query: {query}</p>
        </div>
        <div style="padding:15px;">
            <p style="color:#fff;margin:5px 0;">
                <b>Stats:</b> {len(sem_bm25_overlap)} docs in both Semantic & BM25 (⭐) |
                {total_unique} unique docs across all stages
            </p>
            <p style="color:#888;font-size:11px;margin:5px 0;">
                📊 Semantic | 🔤 BM25 | 🔀 Hybrid (RRF) | 🎯 Rerank (Cross-Encoder)
            </p>
            <div style="overflow-x:auto;margin-top:15px;">
                <table style="width:100%;border-collapse:collapse;min-width:900px;">
                    <tr style="background:#1a1a1a;">
                        <th style="padding:10px;text-align:center;color:#fff;border-bottom:2px solid #3498db;width:50px;">Final</th>
                        <th style="padding:10px;text-align:left;color:#fff;border-bottom:2px solid #3498db;width:120px;">Chunk ID</th>
                        <th style="padding:10px;text-align:center;color:#3498db;border-bottom:2px solid #3498db;width:80px;">📊 Semantic</th>
                        <th style="padding:10px;text-align:center;color:#f39c12;border-bottom:2px solid #f39c12;width:80px;">🔤 BM25</th>
                        <th style="padding:10px;text-align:center;color:#9b59b6;border-bottom:2px solid #9b59b6;width:80px;">🔀 Hybrid</th>
                        <th style="padding:10px;text-align:center;color:#27ae60;border-bottom:2px solid #27ae60;width:80px;">🎯 Rerank</th>
                        <th style="padding:10px;text-align:center;color:#fff;border-bottom:2px solid #3498db;width:50px;">Move</th>
                        <th style="padding:10px;text-align:left;color:#fff;border-bottom:2px solid #3498db;">Text Preview</th>
                    </tr>
                    {rows}
                </table>
            </div>
            <p style="color:#666;font-size:10px;margin-top:10px;">
                Legend: # = rank in that stage | Score badge shows the score value | ⭐ = in both Semantic & BM25 | Move = rank change from Hybrid → Rerank
            </p>
        </div>
    </div>
    """
    display(HTML(html))

## Section 5: Hybrid Search with Reciprocal Rank Fusion

### Why Combine Semantic + Keyword?

Each method has strengths:
- **Semantic**: Understands meaning, synonyms, paraphrasing
- **BM25**: Exact matches, technical terms, IDs

**Reciprocal Rank Fusion (RRF)** combines multiple rankings:
```
RRF_score(d) = Σ 1 / (k + rank_i(d))
```

Where:
- `k` is a smoothing constant (typically 60)
- `rank_i(d)` is the rank of document d in ranking i

Documents that appear high in *both* rankings get the highest combined scores.

In [12]:
def reciprocal_rank_fusion(
    rankings: List[List[Dict]],
    k: int = 60
) -> List[Dict]:
    """
    Combine multiple rankings using Reciprocal Rank Fusion.

    Args:
        rankings: List of ranked result lists (each with 'chunk_id' key)
        k: Smoothing parameter (default 60)

    Returns:
        Combined ranked list with RRF scores
    """
    rrf_scores = {}
    chunk_data = {}

    for ranking in rankings:
        for rank, item in enumerate(ranking, 1):
            chunk_id = item['chunk_id']

            # Store chunk data
            if chunk_id not in chunk_data:
                chunk_data[chunk_id] = item

            # Accumulate RRF score
            if chunk_id not in rrf_scores:
                rrf_scores[chunk_id] = 0

            rrf_scores[chunk_id] += 1.0 / (k + rank)

    # Sort by RRF score
    sorted_ids = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)

    # Build result list
    results = []
    for chunk_id in sorted_ids:
        item = chunk_data[chunk_id].copy()
        item['rrf_score'] = rrf_scores[chunk_id]
        results.append(item)

    return results


class HybridRetriever:
    """
    Hybrid retriever combining BM25 and semantic search with RRF.
    """

    def __init__(self, bm25_retriever: BM25Retriever, collection):
        self.bm25 = bm25_retriever
        self.collection = collection

    def search(self, query: str, top_k: int = 10,
               n_candidates: int = 30) -> List[Dict]:
        """
        Hybrid search combining BM25 and semantic.

        Args:
            query: Search query
            top_k: Number of final results
            n_candidates: Number of candidates from each method

        Returns:
            Combined results with RRF scores
        """
        # Get BM25 results
        bm25_results = self.bm25.search(query, top_k=n_candidates)

        # Get semantic results
        semantic_results = semantic_search(query, self.collection, top_k=n_candidates)

        # Combine with RRF
        combined = reciprocal_rank_fusion([bm25_results, semantic_results])

        return combined[:top_k]


# Create hybrid retriever
hybrid_retriever = HybridRetriever(bm25_retriever, collection)
print("✓ Hybrid retriever created")

✓ Hybrid retriever created


In [27]:
# Compare all three methods - showing how RRF fusion works
test_query = "Workshop aktivnost ocjenjivanje studenata"

print(f"Query: {test_query}")
print("\n" + "="*80)

# Get more results to see overlap
n_show = 10

# Semantic
sem_results = semantic_search(test_query, collection, top_k=n_show)
sem_ids = {r['chunk_id']: i+1 for i, r in enumerate(sem_results)}

# BM25
bm25_results = bm25_retriever.search(test_query, top_k=n_show)
bm25_ids = {r['chunk_id']: i+1 for i, r in enumerate(bm25_results)}

# Find overlap
overlap = set(sem_ids.keys()) & set(bm25_ids.keys())

print(f"\n📊 SEMANTIC SEARCH (top {n_show}):")
for i, r in enumerate(sem_results, 1):
    marker = "⭐" if r['chunk_id'] in overlap else "  "
    print(f"  {marker}[{i}] {r['chunk_id']} | Sim: {r['similarity']:.3f}")

print(f"\n🔤 BM25 KEYWORD SEARCH (top {n_show}):")
for i, r in enumerate(bm25_results, 1):
    marker = "⭐" if r['chunk_id'] in overlap else "  "
    print(f"  {marker}[{i}] {r['chunk_id']} | Score: {r['bm25_score']:.3f}")

print(f"\n⭐ Chunks in BOTH rankings: {len(overlap)}")
for cid in overlap:
    print(f"   {cid}: semantic rank {sem_ids[cid]}, BM25 rank {bm25_ids[cid]}")

# Hybrid with explanation
print(f"\n🔀 HYBRID SEARCH (RRF with k=60):")
hybrid_results = hybrid_retriever.search(test_query, top_k=n_show)
for i, r in enumerate(hybrid_results, 1):
    cid = r['chunk_id']
    sem_rank = sem_ids.get(cid, ">10")
    bm25_rank = bm25_ids.get(cid, ">10")
    in_both = "⭐" if cid in overlap else "  "
    print(f"  {in_both}[{i}] {cid} | RRF: {r['rrf_score']:.4f} (sem:{sem_rank}, bm25:{bm25_rank})")

print("\n" + "="*80)
print("💡 RRF formula: score = Σ 1/(k + rank)")
print("   Chunks appearing in BOTH lists get higher scores!")
print("   ⭐ = appears in both semantic and BM25 top-10")


Query: Workshop aktivnost ocjenjivanje studenata


📊 SEMANTIC SEARCH (top 10):
  ⭐[1] moodle_chunk_27 | Sim: 0.660
    [2] moodle_chunk_28 | Sim: 0.654
  ⭐[3] moodle_chunk_29 | Sim: 0.646
  ⭐[4] moodle_chunk_26 | Sim: 0.623
    [5] tecaj_chunk_113 | Sim: 0.606
    [6] tecaj_chunk_14 | Sim: 0.592
    [7] tecaj_chunk_112 | Sim: 0.582
    [8] tecaj_chunk_13 | Sim: 0.581
  ⭐[9] tecaj_chunk_122 | Sim: 0.578
    [10] moodle_chunk_22 | Sim: 0.578

🔤 BM25 KEYWORD SEARCH (top 10):
  ⭐[1] tecaj_chunk_122 | Score: 7.175
    [2] tecaj_chunk_101 | Score: 6.954
    [3] tecaj_chunk_91 | Score: 6.628
    [4] tecaj_chunk_100 | Score: 6.613
  ⭐[5] moodle_chunk_26 | Score: 6.402
    [6] tecaj_chunk_82 | Score: 6.263
    [7] tecaj_chunk_89 | Score: 6.263
    [8] tecaj_chunk_114 | Score: 6.134
  ⭐[9] moodle_chunk_27 | Score: 6.057
  ⭐[10] moodle_chunk_29 | Score: 5.940

⭐ Chunks in BOTH rankings: 4
   moodle_chunk_26: semantic rank 4, BM25 rank 5
   moodle_chunk_27: semantic rank 1, BM25 rank 9
   tecaj_ch

## Section 6: Cross-Encoder Reranking with Qwen3

### Bi-Encoders vs Cross-Encoders

**Bi-Encoders** (what we've used so far):
- Encode query and documents *separately*
- Compare with cosine similarity
- Fast: can pre-compute document embeddings
- Good for first-stage retrieval (recall)

**Cross-Encoders**:
- Encode query AND document *together*
- Model sees the full interaction
- Slow: must process each pair
- Better accuracy: used for second-stage reranking (precision)

### Qwen3 Reranker

We'll use **Qwen3-Reranker** - a state-of-the-art multilingual reranker that supports 100+ languages including Croatian:

| Model | Size | VRAM | Quality |
|-------|------|------|---------|
| `tomaarsen/Qwen3-Reranker-0.6B-seq-cls` | 0.6B | ~1.2GB | Good |
| `dengcao/Qwen3-Reranker-4B-seq-cls` | 4B | ~8GB | Better |

Both fit on a T4 GPU (16GB). The `-seq-cls` versions are optimized for efficiency.

### Two-Stage Pipeline
```
Stage 1: Retrieve many candidates (20-50) using fast bi-encoder
Stage 2: Rerank top candidates using accurate Qwen3 cross-encoder
```

In [28]:
# Qwen3 Reranker - Multilingual cross-encoder
#
# Two options available:
# 1. seq-cls version (recommended): More efficient, simpler API
# 2. Original version: Uses yes/no logits, more complex but official

# We'll use the seq-cls version for efficiency
RERANKER_MODEL = "tomaarsen/Qwen3-Reranker-0.6B-seq-cls"
# Alternative: "dengcao/Qwen3-Reranker-4B-seq-cls" (better quality, more VRAM)


def format_query_for_qwen3(query: str, instruction: str = None) -> str:
    """Format query with Qwen3 reranker prompt template."""
    prefix = '<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be "yes" or "no".<|im_end|>\n<|im_start|>user\n'
    if instruction is None:
        instruction = "Given a web search query, retrieve relevant passages that answer the query"
    return f"{prefix}<Instruct>: {instruction}\n<Query>: {query}\n"


def format_document_for_qwen3(document: str) -> str:
    """Format document with Qwen3 reranker prompt template."""
    suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
    return f"<Document>: {document}{suffix}"


class Qwen3Reranker:
    """
    Qwen3 cross-encoder reranker for multilingual relevance scoring.

    Supports 100+ languages including Croatian.
    Runs on GPU (T4 compatible).
    """

    def __init__(self, model_name: str = RERANKER_MODEL):
        """
        Initialize Qwen3 reranker.

        Args:
            model_name: HuggingFace model ID
                - tomaarsen/Qwen3-Reranker-0.6B-seq-cls (0.6B, fast)
                - dengcao/Qwen3-Reranker-4B-seq-cls (4B, better)
        """
        print(f"Loading Qwen3 reranker: {model_name}...")
        self.model = CrossEncoder(model_name, trust_remote_code=True)
        print(f"✓ Qwen3 reranker loaded (multilingual, 100+ languages)")

    def rerank(
        self,
        query: str,
        candidates: List[Dict],
        top_k: int = 5,
        instruction: str = None
    ) -> List[Dict]:
        """
        Rerank candidates using Qwen3 cross-encoder.

        Args:
            query: Search query (any language)
            candidates: List of candidate results with 'text' key
            top_k: Number of results to return
            instruction: Optional task-specific instruction

        Returns:
            Reranked list with 'rerank_score' (0-1, higher = more relevant)
        """
        if not candidates:
            return []

        # Format pairs with Qwen3 template
        formatted_query = format_query_for_qwen3(query, instruction)
        pairs = [
            [formatted_query, format_document_for_qwen3(c['text'])]
            for c in candidates
        ]

        # Score with cross-encoder
        scores = self.model.predict(pairs)

        # Add scores to candidates
        for i, candidate in enumerate(candidates):
            candidate['rerank_score'] = float(scores[i])

        # Sort by rerank score (higher = more relevant)
        reranked = sorted(candidates, key=lambda x: x['rerank_score'], reverse=True)

        return reranked[:top_k]


# Create reranker (uses GPU automatically if available)
reranker = Qwen3Reranker()

# Alias for compatibility with rest of notebook
Reranker = Qwen3Reranker

Loading Qwen3 reranker: tomaarsen/Qwen3-Reranker-0.6B-seq-cls...
✓ Qwen3 reranker loaded (multilingual, 100+ languages)


In [29]:
# Demonstrate two-stage retrieval
test_query = "How do students receive grades in workshop?"

print(f"Query: {test_query}")
print("\n" + "="*80)

# Stage 1: Get more candidates with hybrid search
print("\n📥 STAGE 1: Retrieve 10 candidates with hybrid search")
candidates = hybrid_retriever.search(test_query, top_k=10)

print("\nTop 5 before reranking:")
for i, c in enumerate(candidates[:5], 1):
    print(f"  [{i}] {c['chunk_id']} | RRF: {c['rrf_score']:.4f}")
    print(f"      {c['text'][:80]}...")

# Stage 2: Rerank with cross-encoder
print("\n" + "="*80)
print("\n🎯 STAGE 2: Rerank with cross-encoder")
reranked = reranker.rerank(test_query, candidates, top_k=5)

print("\nTop 5 after reranking:")
for i, c in enumerate(reranked, 1):
    print(f"  [{i}] {c['chunk_id']} | Rerank: {c['rerank_score']:.3f}")
    print(f"      {c['text'][:80]}...")

print("\n💡 Notice how reranking may change the order based on query-document relevance!")

Query: How do students receive grades in workshop?


📥 STAGE 1: Retrieve 10 candidates with hybrid search

Top 5 before reranking:
  [1] moodle_chunk_28 | RRF: 0.0323
      will be the same colour.** * **You are ready to switch to the Submission phase w...
  [2] moodle_chunk_29 | RRF: 0.0323
      the Grading evaluation phase. This phase will be highlighted.** #### Grading eva...
  [3] moodle_chunk_26 | RRF: 0.0320
      Quiz navigation block:** ### Teacher view **Teachers should preview the quiz to ...
  [4] moodle_chunk_27 | RRF: 0.0315
      this opens more options.)** 9. **Availability gives you the option to allow stud...
  [5] moodle_chunk_15 | RRF: 0.0303
      could access the course without paying.** # [**Tracking progress - MoodleDocs**]...


🎯 STAGE 2: Rerank with cross-encoder

Top 5 after reranking:
  [1] moodle_chunk_26 | Rerank: 0.999
      Quiz navigation block:** ### Teacher view **Teachers should preview the quiz to ...
  [2] moodle_chunk_27 | Rerank: 0.998
      this

In [30]:
# Complete 4-stage retrieval comparison
test_query = "How do students receive grades in workshop?"

print(f"Query: {test_query}")
print("\n" + "="*90)

n_show = 10

# Stage 1a: Semantic Search
sem_results = semantic_search(test_query, collection, top_k=n_show)
sem_ids = {r['chunk_id']: i+1 for i, r in enumerate(sem_results)}

# Stage 1b: BM25 Search
bm25_results = bm25_retriever.search(test_query, top_k=n_show)
bm25_ids = {r['chunk_id']: i+1 for i, r in enumerate(bm25_results)}

# Stage 2: Hybrid (RRF)
hybrid_results = hybrid_retriever.search(test_query, top_k=n_show)
hybrid_ids = {r['chunk_id']: i+1 for i, r in enumerate(hybrid_results)}

# Stage 3: Rerank
reranked_results = reranker.rerank(test_query, hybrid_results.copy(), top_k=n_show)
rerank_ids = {r['chunk_id']: i+1 for i, r in enumerate(reranked_results)}

# Find overlaps
sem_bm25_overlap = set(sem_ids.keys()) & set(bm25_ids.keys())
all_ids = set(sem_ids.keys()) | set(bm25_ids.keys()) | set(hybrid_ids.keys()) | set(rerank_ids.keys())

print(f"\n📊 STAGE 1a: SEMANTIC SEARCH")
print("-" * 50)
for i, r in enumerate(sem_results[:n_show], 1):
    marker = "⭐" if r['chunk_id'] in sem_bm25_overlap else "  "
    print(f"  {marker}[{i:2d}] {r['chunk_id']:20s} | Sim: {r['similarity']:.3f}")

print(f"\n🔤 STAGE 1b: BM25 KEYWORD SEARCH")
print("-" * 50)
for i, r in enumerate(bm25_results[:n_show], 1):
    marker = "⭐" if r['chunk_id'] in sem_bm25_overlap else "  "
    print(f"  {marker}[{i:2d}] {r['chunk_id']:20s} | Score: {r['bm25_score']:.3f}")

print(f"\n⭐ In BOTH semantic & BM25: {len(sem_bm25_overlap)}")
for cid in list(sem_bm25_overlap)[:5]:
    print(f"   {cid}: sem={sem_ids[cid]}, bm25={bm25_ids[cid]}")

print(f"\n🔀 STAGE 2: HYBRID FUSION (RRF)")
print("-" * 50)
for i, r in enumerate(hybrid_results[:n_show], 1):
    cid = r['chunk_id']
    sem_r = sem_ids.get(cid, "-")
    bm25_r = bm25_ids.get(cid, "-")
    in_both = "⭐" if cid in sem_bm25_overlap else "  "
    print(f"  {in_both}[{i:2d}] {cid:20s} | RRF: {r['rrf_score']:.4f} (sem:{sem_r}, bm25:{bm25_r})")

print(f"\n🎯 STAGE 3: CROSS-ENCODER RERANKING (Qwen3)")
print("-" * 50)
for i, r in enumerate(reranked_results[:n_show], 1):
    cid = r['chunk_id']
    hybrid_r = hybrid_ids.get(cid, "-")
    score = r['rerank_score']
    # Show movement from hybrid rank
    if hybrid_r != "-":
        movement = hybrid_r - i
        if movement > 0:
            move_str = f"↑{movement}"
        elif movement < 0:
            move_str = f"↓{-movement}"
        else:
            move_str = "="
    else:
        move_str = "?"
    score_color = "🟢" if score > 0.5 else "🟡" if score > 0.1 else "🔴"
    print(f"  {score_color}[{i:2d}] {cid:20s} | Rerank: {score:.3f} (was #{hybrid_r}) {move_str}")

print("\n" + "="*90)
print("💡 Legend: ⭐=in both sem+bm25 | 🟢>0.5 | 🟡>0.1 | 🔴<0.1 | ↑↓=rank change from hybrid")

Query: How do students receive grades in workshop?


📊 STAGE 1a: SEMANTIC SEARCH
--------------------------------------------------
  ⭐[ 1] moodle_chunk_28      | Sim: 0.729
  ⭐[ 2] moodle_chunk_29      | Sim: 0.700
  ⭐[ 3] moodle_chunk_27      | Sim: 0.684
  ⭐[ 4] moodle_chunk_26      | Sim: 0.616
  ⭐[ 5] moodle_chunk_22      | Sim: 0.590
  ⭐[ 6] moodle_chunk_1       | Sim: 0.576
  ⭐[ 7] moodle_chunk_15      | Sim: 0.566
    [ 8] tecaj_chunk_112      | Sim: 0.554
    [ 9] tecaj_chunk_113      | Sim: 0.553
    [10] tecaj_chunk_13       | Sim: 0.539

🔤 STAGE 1b: BM25 KEYWORD SEARCH
--------------------------------------------------
  ⭐[ 1] moodle_chunk_26      | Score: 20.991
  ⭐[ 2] moodle_chunk_29      | Score: 18.284
  ⭐[ 3] moodle_chunk_28      | Score: 17.311
  ⭐[ 4] moodle_chunk_27      | Score: 16.956
  ⭐[ 5] moodle_chunk_15      | Score: 10.511
    [ 6] moodle_chunk_48      | Score: 9.955
  ⭐[ 7] moodle_chunk_22      | Score: 9.841
  ⭐[ 8] moodle_chunk_1       | Score: 9.806
   

## Section 7: Confidence Thresholds

### When to Say "I Don't Know"

A good RAG system should:
- Answer confidently when it has relevant information
- Decline gracefully when it doesn't

We can use the rerank score as a confidence measure:
- High score → relevant documents found → generate answer
- Low score → no relevant documents → decline to answer

In [16]:
from IPython.display import display, HTML

def query_with_confidence(
    question: str,
    hybrid_retriever: HybridRetriever,
    reranker,
    threshold: float = 0.0,
    top_k: int = 5
) -> Dict[str, Any]:
    """
    Query with confidence threshold.
    """
    # Stage 1: Hybrid retrieval
    candidates = hybrid_retriever.search(question, top_k=20)

    # Stage 2: Rerank
    reranked = reranker.rerank(question, candidates, top_k=top_k)

    if not reranked:
        return {
            'confident': False,
            'max_score': 0.0,
            'results': [],
            'message': 'No results found.'
        }

    max_score = reranked[0]['rerank_score']

    if max_score < threshold:
        return {
            'confident': False,
            'max_score': max_score,
            'results': reranked,
            'message': f'Confidence too low ({max_score:.2f} < {threshold}). I may not have relevant information.'
        }

    return {
        'confident': True,
        'max_score': max_score,
        'results': reranked,
        'message': 'Found relevant information.'
    }




# Test with in-domain question
in_domain = "Kako nastavnik može izraditi sigurnosnu kopiju tečaja?"
result_in = query_with_confidence(in_domain, hybrid_retriever, reranker, threshold=0.0)
display_results(in_domain, result_in)

# Test with out-of-domain question
out_domain = "What is the best pizza restaurant in Zagreb?"
result_out = query_with_confidence(out_domain, hybrid_retriever, reranker, threshold=0.5)
display_results(out_domain, result_out)

#,Score,Source,Full Text
1,1.000,tecaj.mdtecaj_chunk_126,"ako iste postavke želite koristiti i u novom tečaju. Takva je postavka primjerice, datum početka nastave. Odaberite one aktivnosti i resurse iz sekcija koje želite obnoviti u tečaju i odaberite Nastavi U Pregledu su prikazani su svi elementi koje ste odabrali za obnovu u novom tečaju. Ako ste zadovoljni odabirom elemenata odaberite Pokreni postupak vraćanja. Prilikom postupka raspakiranja tečaja mogu se pojaviti i određena upozorenja kao rezutlat različitih verzija sustava. U posljednjem koraku Završeno sustav prikazuje poruku o uspješno kreiranoj sigurnosnoj kopiji. Sakrij transkript https://usavrsavanje.loomen.carnet.hr/mod/page/view.php?id=106961&lang=hr,""Moodle dizajner\_1: Upute za preuzimanje demo tečaja | usavrsavanje Pokaži transkript Da bi izradili sigurnosnu kopiju tečaja, unutar bloka Postavke -”Administracija kolegija” odaberite Backup. U vrhu otvorenog sučelja nalazi se navigacijska traku za izradu sigurnosne kopije u nekoliko koraka: U prvom kotraku, Inicijalne postavke klikom na kućicu odaberite glavne elemente koje želite u sigurnosnoj kopiji –korisničke podatke, aktivnosti, blokove, filtre i ostalo. Iz sigurnosnih razloga na vašem demo tečaju s ulogom nastavnika niste u mogućnosti pohraniti korisničke podatke polaznika tečaja. IMS Common Cartridge (IMS paket) omogućava izradu sigurnosne kopije napravljene u skladu s IMS specifikacijom. te takvu sigurnosnu kopiju možete unijeti i raspakirati u novi Moodle tečaj pomoću resursa IMS paket. Nakon što ste odabrali elemente koje želite pohraniti, odaberite gumb Nastavi . Unutar Postavke sheme - sustav će automatski predložiti uključivanje svih aktivnosti i resursa u sigurnosnu kopiju. Ako ne želite pohraniti sve odabrane podatke, listu prilagodite na način da pojedini element ili cijelu temu isključite klikom na kvačicu pored njihovog naziva. U desnom stupcu prikazano je pohranjivane korisničkih informacija za odabrane aktivnosti i resurse. Obzirom da je u prvom koraku iskljućena mogućnost pohrane korisničkih podataka, i ovdje će svi podaci biti automatski isključeni. Odaberite gumb Nastavi. Slijedi “potvrda i pregled” gdje je budućem .mbz dokumentu sustav već dodijelio naziv koji možete promijeniti jednostavnim upisom"
2,1.000,tecaj.mdtecaj_chunk_123,"akcija od prvog pristupanja polaznika tečaju. Svi zapisi Skrećeni izvještaj - prikazuje popis resursa i aktivnosti kojima je polaznik pristupao i njihov broj, prikazano po temama. Skraćeni izvještaj Potpuni izvještaj - prikazuje detaljniji prikaz resursa i aktivnosti kojima je polaznik pristupao, prikaz po temama. Potpuni izvještaj Statistika – prikazuje statistički i brojčani prikaz aktivnosti, pregleda i postova, za odabranog sudionika. Statistika Ocjena - svaki sudionik vidi svoj izvještaj o ocjenama - prikaz ocjena po aktivnostima, opseg ocjene, postotak ostavarene ocjene u odnou na maksimalnu moguću te prikaz komentara nastavnika na predane zadatke i testove. https://usavrsavanje.loomen.carnet.hr/mod/book/tool/print/index.php?id=106958&chapterid=41087,""Rad s polaznicima | usavrsavanje Pokaži transkript U lijevom bočnom izborniku u bloku “Postavke” pod Administracija e-kolegija odaberite opciju Korisnici pa potom Upisani korisnici. Sučelje koje nam se otvorilo nam prikazuje popis korisnika u tečaju. Klikom na gumb Upiši korisnike u gornjem desnom ili donjem desnom uglu otvara nam se prozor gdje pretražujemo postojeće korisnike u sustavu te odabirom željenih unosimo ih u sustav. Obratite pozornost da na vrhu prozora u padajućem izborniku definirate ulogu koju će korisnik imati pri upisu. Nakon odabira korisnika kliknemo na Završi sa upisivanjem korisnika. Sa postojećeg popisa korisnika možemo vidjeti informacije o samom korisniku, njegov zadnji pristup tečaju, uloge koje ima, te način upisa. Klikom na oznaku X u desnom dijelu popisa korisnika brišemo iz tečaja. Sakrij transkript https://moodle.com/cloud/ Pokaži transkript Da bi izradili sigurnosnu kopiju te

#,Score,Source,Full Text
1,0.000,moodle.mdmoodle_chunk_60,|
2,0.000,tecaj.mdtecaj_chunk_111,"cijelog pitanja. Sustav omogućava određivanje mjernih jedinica te utjecaj postavljanja ili nepostavljanja mjerne jedinice u odgovoru. Da bi sustav Moodle prepoznao postojanje varijabli potrebno je kliknuti na Pronađi zamjenske znakove. Zamjenski znakovi Nakon toga odredite koliko će mogućih promjenjivih varijabli imati svako pitanje: Broj promjenjivih varijabli Kada definirate određeni broj promjenjivih varijabli preko gumba Prikaži možete vidjeti ponuđene vrijednosti te ih promjeniti ovisno o željama i potrebama. Rješenja Pitanje Točno/Netočno Pitanje Točno/Netočno sastoji se od dva moguća odgovora: Točno ili Netočno. U tekst pitanja upišite tvrdnju a u polju Točan odgovor iz padajućeg izbornika odaberite da li je tvrdnja točna ili nije. Dodatno, imate mogućnost upisivanja povratne informacije polaznicima za točan i netočan odgovor. Točno/Netočno Pitanje s umetanjem riječi koje nedostaju (Cloze) Pitanje s umetanjem riječi koje nedostaju specifična je vrsta pitanja koja omogućava umetanje polaznikovih odgovora izravno u tekst. Da bi to bilo moguće potrebno je tekst pitanja napisati u posebnom formatu. Pitanja mogu sadržavati kratke odgovore, višestruki odabir ili numerički dio pitanja. Formula za kratki odgovor na pitanje Koji je glavni grad Hrvatske izgleda ovako: {1:SHORTANSWER:=Zagreb} je glavni grad Hrvatske, pri čemu broj 1 označava broj bodova koji će se pridružiti pitanju. Cloze vrsta pitanja Na sustav smo dodali dodatak za lakše kreiranje ove vrste pitanja jer je izrada dosta komplcirana. Ukoliko se klikne na gumb Cloze uređivač prikazat će se novi prozor u kojem možemo odabrati koju vrstu odgovora želimo dobiti od naših polaznika. Ukoliko označimo vrstu odgovora s desne strane ćemo dobiti objašnjenje vrste pitanja koju smo odabrali. Cloze uređivač Pomoću gumba Dekodiraj i provjeri tekst pitanja možete provjeriti da li je pitanje napisano u ispravno. Ako jest, sustav će dodati prikazati sljedeći prozor u postavkama pitanja: Cloze odgovor Upravljanje ocjenama U bloku Administracija kolegija nalazi se link Ocjene . Njegovim odabirom otvara se tablica s"
3,0.000,moodle.mdmoodle_chunk_38,"[**Front page**](https://docs.moodle.org/500/en/Front_page) **- how best to display the entry page to your Moodle. Here you can decide how to display announcements and courses (if you want to) and how to add attractive images and descriptive text to promote your organisation.** * [**Theme settings**](https://docs.moodle.org/500/en/Theme_settings) **- how to add custom dropdown menus and (if you wish) personalise the** [**user menu**](https://docs.moodle.org/500/en/User_menu) **top right.** * [**Header and footer**](https://docs.moodle.org/500/en/Header_and_footer) **- how to display useful information at the top and bottom of your course and link to Google Analytics.** * [**Themes**](https://docs.moodle.org/500/en/Themes) **- general information on changing the ""skin"" of your Moodle for the whole site or just sections. Explore** [**Standard themes**](https://docs.moodle.org/500/en/Standard_themes)**.** * [**Dashboard**](https://docs.moodle.org/500/en/Dashboard) **- a personalised page for each user. Find out how to set the default dashboard and how to allow users to select their own home page.** * [**User profiles**](https://docs.moodle.org/500/en/User_profiles) * [**Navigation**](https://docs.moodle.org/500/en/Navigation) **- control how users find their way around Moodle** * [**Course list**](https://docs.moodle.org/500/en/Course_list) **- control who appears in the list of courses** # [**Installing plugins - MoodleDocs**](https://docs.moodle.org/500/en/Installing_plugins) ## Why install additional plugins? **Plugins enable you to add additional features and functionality to Moodle, such as new activities, new quiz question types, new reports, integrations with other systems and many more.** **Note: Certain hosting solutions, such as** [**Mood

## Section 8: Source Citations

### Proper Attribution

Good RAG answers should:
- Cite which documents were used
- Allow users to verify the information
- Build trust in the system

In [17]:
# Test out different LLM answer models here
LLM_MODEl = "anthropic/claude-sonnet-4.5"

In [18]:
CITATION_PROMPT_TEMPLATE = """Ti si pametni asistent koji odgovara na pitanja koristeći dostavljeni kontekst.

KONTEKST:
{context}

PITANJE: {question}

UPUTE:
1. Koristi SAMO informacije iz gornjeg konteksta
2. Za svaku tvrdnju navedi izvor u formatu [Izvor X]
3. Ako informacija nije u kontekstu, reci "Nemam tu informaciju u dostavljenim dokumentima"
4. Budi koncizan ali potpun
5. Na kraju odgovora napiši popis korištenih izvora

ODGOVOR:"""


def generate_answer_with_citations(
    question: str,
    results: List[Dict],
    model: str = LLM_MODEL
) -> str:
    """
    Generate answer with source citations.
    """
    # Build context with source labels
    context_parts = []
    for i, r in enumerate(results, 1):
        source = r['metadata']['source']
        context_parts.append(f"[Izvor {i} - {source}]:\n{r['text']}")

    context = "\n\n".join(context_parts)

    prompt = CITATION_PROMPT_TEMPLATE.format(
        context=context,
        question=question
    )

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=1000
    )

    return response.choices[0].message.content


# Test citation generation with nice display
test_question = "Koje su faze Workshop aktivnosti?"
search_result = query_with_confidence(test_question, hybrid_retriever, reranker)

if search_result['confident']:
    answer = generate_answer_with_citations(test_question, search_result['results'][:3])
    display_results(test_question, search_result, show_answer=answer)
else:
    display_results(test_question, search_result)

#,Score,Source,Full Text
1,0.996,moodle.mdmoodle_chunk_26,"Quiz navigation block:** ### Teacher view **Teachers should preview the quiz to ensure it displays as desired for students:** **Grades can be viewed either by clicking the quiz and the link 'Attempts' when students have attempted the quiz, or from the Actions menu top right > Results (as in the above screenshot)** # [**Workshop activity - MoodleDocs**](https://docs.moodle.org/500/en/Workshop_activity) ## What is the Workshop activity? **Workshop is a powerful peer assessment activity. Students add submissions which are then distributed amongst their peers for assessment based on a grading scale specified by the teacher.** ## How is it set up? 1. **In a course, with the editing turned on, choose 'Workshop' from the activity chooser.** 2. **Give it a name and, if needed, a description** 3. **Expand the other sections to select the settings you want. If you are not sure, leave everything as default. See** [**Workshop settings**](https://docs.moodle.org/500/en/Workshop_settings) **for more detailed information.** 4. **Grading settings - students receive two grades, one for the work they submit and one for the quality of their peer assessments. The page** [**Workshop grading strategies**](https://docs.moodle.org/500/en/Workshop_grading_strategies) **explains this more.** 5. **Submission settings is where you explain the task they must submit.** 6. **Assessment settings is where you give a brief outline of how they will assess the work of their peers.** 7. **Feedback will, if enabled, allow students to add text comments when they review each other's work.** 8. **Example submissions, if enabled, allows you to provide examples for students to practise with before they begin peer assessing. (Enabling this opens more options.)** 9. **Availability gives you the option to allow students to start peer assessing as soon as the submission deadline is over, rather than you enabling this manually.** 10. **Click Save and display and explore the Workshop phases in the section Teacher view below, making sure you"
2,0.995,moodle.mdmoodle_chunk_27,"this opens more options.)** 9. **Availability gives you the option to allow students to start peer assessing as soon as the submission deadline is over, rather than you enabling this manually.** 10. **Click Save and display and explore the Workshop phases in the section Teacher view below, making sure you complete the Set up phase and switch to the Submission phase when you want your students to begin the activity.** ## How does it work? ### Student view * **Students click on the Workshop icon in the course to access it. Initially they will see either the Set up phase (if the teacher is still preparing the Workshop) or the Submission phase when they are allowed to submit.** * **Other phases will be highlighted at different times depending on the teacher's settings. During the Assessment phase, students scroll down the page to view the submissions they must review:** * **Once grades are calculated and finalised, and the workshop is closed, students see their two grades.** ### Teacher view **Once a Workshop activity has been created and saved, it is in the Set up phase. It must be in the Submission phase for students to be able to submit work and then moved to the Assessment phase for them to review each other's submissions. The switch may automatic or manual.** #### Setup phase * **Click 'Edit assessment form' to provide detailed grading criteria for your students to use. When finished, click 'Save and close', and all ticks on the Setup phase will be the same colour.** * **You are ready to switch to the Submission phase which lets students send in their work. Click the icon or text at the top of Submission phase. This phase will be highlighted.** #### Submission phase * **Students will now be able to submit their"
3,0.944,moodle.mdmoodle_chunk_28,"will be the same colour.** * **You are ready to switch to the Submission phase which lets students se

## Section 9: Complete Enhanced RAG Pipeline

Now let's put it all together into a single class.

In [19]:
@dataclass
class RAGConfig:
    """Configuration for Enhanced RAG Pipeline."""
    top_k_retrieve: int = 20
    top_k_rerank: int = 5
    confidence_threshold: float = 0.0
    use_hybrid: bool = True
    use_reranking: bool = True
    llm_model: str = LLM_MODEL


class EnhancedRAGPipeline:
    """
    Complete Enhanced RAG Pipeline with:
    - Hybrid search (semantic + BM25)
    - Cross-encoder reranking
    - Confidence thresholds
    - Source citations
    """

    def __init__(
        self,
        collection,
        chunks: List[Dict],
        config: RAGConfig = None
    ):
        self.config = config or RAGConfig()
        self.collection = collection

        # Build retrievers
        self.bm25_retriever = BM25Retriever(chunks)
        self.hybrid_retriever = HybridRetriever(self.bm25_retriever, collection)

        # Build reranker
        if self.config.use_reranking:
            self.reranker = Reranker()
        else:
            self.reranker = None

        print("✓ Enhanced RAG Pipeline initialized")
        print(f"  - Hybrid search: {self.config.use_hybrid}")
        print(f"  - Reranking: {self.config.use_reranking}")
        print(f"  - Confidence threshold: {self.config.confidence_threshold}")

    def retrieve(self, question: str) -> List[Dict]:
        """
        Retrieve relevant documents.
        """
        # Stage 1: Retrieval
        if self.config.use_hybrid:
            candidates = self.hybrid_retriever.search(
                question,
                top_k=self.config.top_k_retrieve
            )
        else:
            candidates = semantic_search(
                question,
                self.collection,
                top_k=self.config.top_k_retrieve
            )

        # Stage 2: Reranking
        if self.config.use_reranking and self.reranker:
            results = self.reranker.rerank(
                question,
                candidates,
                top_k=self.config.top_k_rerank
            )
        else:
            results = candidates[:self.config.top_k_rerank]

        return results

    def query(self, question: str) -> Dict[str, Any]:
        """
        Full RAG query with answer generation.
        """
        # Retrieve
        results = self.retrieve(question)

        if not results:
            return {
                'answer': 'Nema pronađenih rezultata.',
                'confident': False,
                'sources': []
            }

        # Check confidence
        max_score = results[0].get('rerank_score', results[0].get('similarity', 0))
        confident = max_score >= self.config.confidence_threshold

        if not confident:
            return {
                'answer': f'Pouzdanost je preniska ({max_score:.2f}). Nemam dovoljno relevantnih informacija.',
                'confident': False,
                'max_score': max_score,
                'sources': [r['metadata'] for r in results]
            }

        # Generate answer with citations
        answer = generate_answer_with_citations(
            question,
            results,
            model=self.config.llm_model
        )

        return {
            'answer': answer,
            'confident': True,
            'max_score': max_score,
            'sources': [r['metadata'] for r in results],
            'retrieved_chunks': [r['text'] for r in results]
        }

In [20]:
# Create enhanced pipeline
config = RAGConfig(
    top_k_retrieve=15,
    top_k_rerank=3,
    confidence_threshold=0.0,
    use_hybrid=True,
    use_reranking=True
)

enhanced_rag = EnhancedRAGPipeline(collection, all_chunks, config)

✓ BM25 index built with 199 documents
Loading Qwen3 reranker: tomaarsen/Qwen3-Reranker-0.6B-seq-cls...
✓ Qwen3 reranker loaded (multilingual, 100+ languages)
✓ Enhanced RAG Pipeline initialized
  - Hybrid search: True
  - Reranking: True
  - Confidence threshold: 0.0


In [21]:
# Test the complete pipeline
questions = [
    "Kako nastavnik može promijeniti nazive uloga u tečaju?",
    "What are the five phases of a Workshop activity?",
    "Što je opcija 'Ocjenjivanje naslijepo'?"
]

for q in questions:
    print("\n" + "="*80)
    print(f"QUESTION: {q}")
    print("="*80)

    result = enhanced_rag.query(q)

    print(f"\nConfident: {result['confident']}")
    if 'max_score' in result:
        print(f"Max Score: {result['max_score']:.3f}")
    print(f"\nANSWER:")
    print(result['answer'])
    print(f"\nSources: {result['sources']}")


QUESTION: Kako nastavnik može promijeniti nazive uloga u tečaju?

Confident: True
Max Score: 1.000

ANSWER:
# Promjena naziva uloga u tečaju

Nastavnik može promijeniti nazive uloga na sljedeći način:

**Unutar postavki tečaja, u dijelu "Preimenovanje uloga"** nastavnik može preimenovati postojeće uloge te im dodijeliti nove nazive [Izvor 1, Izvor 3].

## Primjer preimenovanja:
- Ulogu **Nastavnika** može preimenovati u **Predavača**
- Ulogu **Studenta** može preimenovati u **Učenika** [Izvor 1, Izvor 3]

## Važna napomena:
**Mijenjanjem naziva uloga ne mijenjaju se njihove ovlasti u tečaju** [Izvor 1, Izvor 3]. To znači da promjena naziva utječe samo na prikaz imena uloge, ali ne i na prava i mogućnosti koje ta uloga ima.

---

**Korišteni izvori:**
- Izvor 1 - tecaj.md
- Izvor 3 - tecaj.md

Sources: [{'source': 'tecaj.md', 'language': 'hr'}, {'source': 'tecaj.md', 'language': 'hr'}, {'source': 'tecaj.md', 'language': 'hr'}]

QUESTION: What are the five phases of a Workshop activity?

## Section 10: Compare Basic vs Enhanced RAG

In [22]:
# Create a basic RAG (no hybrid, no reranking) for comparison
basic_config = RAGConfig(
    top_k_retrieve=5,
    top_k_rerank=5,
    use_hybrid=False,
    use_reranking=False
)

# We need to recreate without reranker
class BasicRAGPipeline:
    def __init__(self, collection, config):
        self.collection = collection
        self.config = config

    def query(self, question: str) -> Dict:
        results = semantic_search(question, self.collection, top_k=self.config.top_k_retrieve)

        if not results:
            return {'answer': 'No results', 'sources': []}

        answer = generate_answer_with_citations(question, results)
        return {
            'answer': answer,
            'sources': [r['metadata'] for r in results]
        }

basic_rag = BasicRAGPipeline(collection, basic_config)

# Compare on a challenging query
comparison_query = "loomen.carnet.hr sigurnosna kopija AAI@Edu.hr"

print(f"Query: {comparison_query}")
print("\n" + "="*80)
print("BASIC RAG (semantic only):")
print("="*80)
basic_result = basic_rag.query(comparison_query)
print(basic_result['answer'][:500] + "...")

print("\n" + "="*80)
print("ENHANCED RAG (hybrid + reranking):")
print("="*80)
enhanced_result = enhanced_rag.query(comparison_query)
print(enhanced_result['answer'][:500] + "...")

Query: loomen.carnet.hr sigurnosna kopija AAI@Edu.hr

BASIC RAG (semantic only):
# Odgovor

Na temelju dostavljenog konteksta, **korisnici s AAI@Edu.hr elektroničkim identitetom mogu koristiti sigurnosnu kopiju svog demo tečaja na sustavu loomen.carnet.hr** na sljedeći način:

## Postupak:

1. **Napravite sigurnosnu kopiju** vašeg demo tečaja slijedeći "Upute za preuzimanje demo tečaja" [Izvor 1, 2, 3, 4, 5]

2. **Prijavite se na sustav https://loomen.carnet.hr** koristeći Vaš AAI@Edu.hr elektronički identitet [Izvor 1, 2, 3, 4, 5]

3. **Ispunite Zahtjev za novim tečajem** -...

ENHANCED RAG (hybrid + reranking):
# Odgovor

Na temelju dostavljenog konteksta, **korisnici s AAI@Edu.hr elektroničkim identitetom** mogu koristiti sigurnosnu kopiju svog demo tečaja na sustavu **loomen.carnet.hr** na sljedeći način:

## Postupak:

1. **Napravite sigurnosnu kopiju** vašeg demo tečaja slijedeći "Upute za preuzimanje demo tečaja" [Izvor 1, 2, 3]

2. **Prijavite se na sustav https://loomen.carnet

## Section 11: Exercises

Try these exercises to deepen your understanding:

1. **Tune the RRF parameter**: Try different values of `k` (30, 60, 100) and see how it affects results
2. **Compare reranker models**: Try `cross-encoder/ms-marco-MiniLM-L-12-v2` vs the L-6 version
3. **Adjust confidence thresholds**: Find the optimal threshold for your use case
4. **Test edge cases**: Try queries in mixed languages, very specific technical terms
5. **Add your own documents**: Extend the pipeline with your own content

In [23]:
# Playground - try your own queries!
your_query = "Kako podesiti da svaki student vidi pitanja drugačijim redoslijedom?"

result = enhanced_rag.query(your_query)

print(f"Query: {your_query}")
print(f"\nConfident: {result['confident']}")
print(f"\nAnswer:\n{result['answer']}")

Query: Kako podesiti da svaki student vidi pitanja drugačijim redoslijedom?

Confident: True

Answer:
Da biste podesili da svaki student vidi pitanja drugačijim redoslijedom, trebate koristiti opciju **Karakteristike pitanja** prilikom kreiranja testa.

Unutar Karakteristika pitanja možete podesiti da se izmiješa redoslijed pitanja. Kada odaberete ovu opciju, pitanja će kod svakog polaznika biti drugačije raspoređena, kao i u slučaju da isti polaznik drugi puta pristupi testu [Izvor 1, Izvor 2].

Dodatno, unutar istih postavki možete podesiti da se osim redoslijeda pitanja izmiješa i **redoslijed odgovora unutar pitanja**. Ako odaberete da odgovori budu izmiješani slučajnim rasporedom, oni će kod svakog polaznika biti drugačije raspoređeni [Izvor 1, Izvor 2].

Ova opcija je korisna ako želite smanjiti mogućnost varanja na testovima [Izvor 1, Izvor 2].

---
**Korišteni izvori:** Izvor 1, Izvor 2


## Summary

### What We Built

1. **BM25 Keyword Search**
   - When exact matches matter
   - Technical terms, IDs, proper nouns

2. **Hybrid Search with RRF**
   - Combines semantic + keyword strengths
   - Reciprocal Rank Fusion for score combination

3. **Cross-Encoder Reranking with Qwen3**
   - Two-stage retrieval pipeline
   - Multilingual support (100+ languages including Croatian)
   - Runs locally on T4 GPU

4. **Confidence Thresholds**
   - Know when to decline answering
   - Avoid hallucinations on out-of-domain queries

5. **Source Citations**
   - Proper attribution in answers
   - Verifiable responses

### Key Takeaways

- **Hybrid > Pure Semantic**: Combining methods handles more query types
- **Reranking improves precision**: Trade speed for accuracy on final results  
- **Qwen3 is multilingual**: Works great for Croatian and English
- **Confidence matters**: Know your system's limitations
- **Citations build trust**: Let users verify your answers

### Next: Notebook 06

In the next notebook, we'll learn how to properly **evaluate** our RAG system:
- Retrieval metrics (Precision@K, Recall@K, MRR)
- LLM-as-Judge for answer quality
- Configuration comparison framework